In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import xlrd
import os
import logging

# Configurar o logger
logFileName = datetime.now().strftime("%Y-%m-%d %H-%M-%S") + ".log"

logging.basicConfig(
    filename=logFileName,  # Nome do arquivo de log
    level=logging.INFO,                # Nível de log (INFO, DEBUG, ERROR, etc.)
    format="%(asctime)s - %(levelname)s - %(message)s",  # Formato das mensagens
)

caminho_arquivo = 'Extrato.xlsx'

if not os.path.exists(caminho_arquivo):
    erro = f"O arquivo {caminho_arquivo} não foi encontrado."
    logging.error(erro)
    raise ValueError(erro)

df = pd.read_excel(caminho_arquivo, header=None)
# Verifica qual linha pode conter os nomes das colunas

if df.empty:
    erro = f"O arquivo {caminho_arquivo} está vazio."
    logging.error(erro)
    raise ValueError(erro)

nomes_colunas = ["Data", "Lançamento", "Valor (R$)", "Saldo (R$)" ]  # Substitua pelos nomes que você conhece
# Procura pela linha que contém todos os nomes das colunas
linha_colunas = -1
for i, linha in df.iterrows():
    if all(nome in linha.values for nome in nomes_colunas):  # Verifica se todos os nomes estão na linha
        linha_colunas = i
        break

if linha_colunas == -1: 
    erro = f"{caminho_arquivo}: Nenhuma linha contendo os nomes das colunas foi encontrada."
    logging.error(erro)
    raise ValueError(erro)

texto = f"{caminho_arquivo}: A linha {linha_colunas} contém os nomes das colunas."
logging.info(texto)
print(texto)

# Recarrega o DataFrame usando essa linha como cabeçalho
df_com_cabecalho = pd.read_excel(caminho_arquivo, header=linha_colunas)

colunas_para_remover = [0]
df_modificado = df_com_cabecalho.drop(df_com_cabecalho.columns[colunas_para_remover], axis=1)
novo_nome = "Notas Fiscais"
df_modificado.rename(columns={df_modificado.columns[-1]: novo_nome}, inplace=True)
df_modificado


In [ ]:
caminho_servicos = 'Acompanhamentos de Serviços.xls'

if not os.path.exists(caminho_servicos):
    erro = f"O arquivo {caminho_servicos} não foi encontrado."
    logging.error(erro)
    raise ValueError(erro)

df_servicos = pd.read_excel(caminho_servicos, header=None)

if df.empty:
    erro = f"O arquivo {caminho_servicos} está vazio."
    logging.error(erro)
    raise ValueError(erro)

nomes_colunas_servicos = ["Código", "Data", "Nota", "Série", "Espécie", "Código", "Cliente", "AC.", "UF", "Valor Contábil", "Tipo", "Base Cálculo", "Alíq.", "Valor", "Isentas", "Outras" ]  # Substitua pelos nomes que você conhece

# Procura pela linha que contém todos os nomes das colunas
linha_colunas_servicos = -1
for i, linha in df_servicos.iterrows():
    if all(nome in linha.values for nome in nomes_colunas_servicos):  # Verifica se todos os nomes estão na linha
        linha_colunas_servicos = i
        break

if linha_colunas_servicos == -1: 
    erro = f"{caminho_servicos}: Nenhuma linha contendo os nomes das colunas foi encontrada."
    logging.error(erro)
    raise ValueError(erro)

texto = f"{caminho_servicos}: A linha {linha_colunas_servicos} contém os nomes das colunas."
logging.info(texto)
print(texto)

nomes_colunas_servicos_2 = ["Nota", "Cliente"]
# Recarrega o DataFrame usando essa linha como cabeçalho
df_servicos_filtrado = pd.read_excel(caminho_servicos, header=linha_colunas_servicos, usecols=nomes_colunas_servicos_2)
df_servicos_filtrado = df_servicos_filtrado.dropna(subset=["Nota"])
df_servicos_filtrado["Nota"] = df_servicos_filtrado["Nota"].astype(int)
df_servicos_filtrado

In [ ]:

extrato = []

for index, row in df_modificado.iterrows():
    texto = row['Notas Fiscais']
    data = row['Data']
    valor = row['Valor (R$)']
    
    texto_limpo = re.sub(r"\s*(\d)\s+(\d)\s*", r"\1\2", texto, flags=re.IGNORECASE)  # Remove espaços entre números
    texto_limpo = re.sub(r"NFS-?e(\d+)", r"NFS-e \1", texto_limpo, flags=re.IGNORECASE)  # Adiciona espaço após "NFS-e" quando seguido por números
    texto_limpo = re.sub(r"\s+", " ", texto_limpo).strip()        # Limpa espaços extras
    
    padrao = r"nfs-?e\s+?(\d+)"
    numeros_notas = re.findall(padrao, texto_limpo, flags=re.IGNORECASE)
    lista_inteiros = [int(x) for x in numeros_notas if x is not None]  # Ignora valores None
    
    if data == '':
        erro = f"{caminho_arquivo}: Nenhuma data encontrada para o lançamento {index}"
        logging.error(erro)
        raise ValueError(erro)

    dataStr = datetime.strftime(data, '%m/%d/%Y')

    if valor == '':
        erro = f"{caminho_arquivo}: Nenhuma valor encontrado para o lançamento {index}"
        logging.error(erro)
        raise ValueError(erro)

    if len(lista_inteiros) == 0:
        erro = f"{caminho_arquivo}: Nenhuma nota fiscal encontrada para o lançamento {index}, {texto}, {texto_limpo}"
        logging.error(erro)
        raise ValueError(erro)
    
    extrato.append({
            'data': dataStr,
            'valor': valor,
            'notas': lista_inteiros,
            'nfs': "NF N " + str(lista_inteiros[0]) + "".join([", " + str(num) for num in lista_inteiros[1:]])
        })


extrato

In [43]:
extrato_conciliado = []
for ext in extrato:
    df_result = df_servicos_filtrado[df_servicos_filtrado["Nota"].isin(ext["notas"])]
    if df_result.empty:
        erro = f"{caminho_arquivo}: Nenhuma nota no arquivo Acompanhamentos de Serviços foi encontrada. Dados do extrado: data={ext['data']}, notas={ext['notas']}, linha={ext['valor']}"
        logging.warning(erro)
    else:
        ext["cliente"] = df_result['Cliente'].values[0]
        extrato_conciliado.append(ext)
        
pd.DataFrame(extrato_conciliado).to_csv('extrato_conciliado.csv', index=False)

In [ ]:
# Texto contendo as notas fiscais
texto = "nfs-e 05091 e nfse 05092 e NFS-e  0    30612	"


texto_limpo = re.sub(r"\s*(\d)\s+(\d)\s*", r"\1\2", texto, flags=re.IGNORECASE)  # Remove espaços entre números
texto_limpo = re.sub(r"NFS-?e(\d+)", r"NFS-e \1", texto_limpo, flags=re.IGNORECASE)  # Adiciona espaço após "NFS-e" quando seguido por números
texto_limpo = re.sub(r"\s+", " ", texto_limpo).strip()        # Limpa espaços extras
# Exibir o resultado
print(texto_limpo)